In [3]:
import numpy as np
import matplotlib.pyplot as plt
import qiskit as qk
import random
from collections import Counter

from generator import generateCliffordCircuit

In [4]:
def stringToQiskitSingleGate(gateString, qiskitCir, whichQubit):
    if gateString == 'I':
        qiskitCir.id(whichQubit)
    elif gateString == 'X':
        qiskitCir.x(whichQubit)
    elif gateString == 'Y':
        qiskitCir.y(whichQubit)
    elif gateString == 'Z':
        qiskitCir.z(whichQubit)
    elif gateString == 'H':
        qiskitCir.h(whichQubit)
    elif gateString == 'S':
        qiskitCir.s(whichQubit)
        
def transpileListToQiskitCircuit(cir):
    depth = len(cir)
    width = len(cir[0])
    qiskitCir = qk.QuantumCircuit(width)
    for d in range(width):
        if d % 2 == 0:
            for w in range(width):
                singleGate = cir[d][w]
                stringToQiskitSingleGate(singleGate, qiskitCir, w)
            if d != width - 1:
                qiskitCir.barrier()
        else:
            c = cir[d].index('CNOT_C')
            t = cir[d].index('CNOT_T')
            qiskitCir.cx(c, t)
            if d != width - 1:
                qiskitCir.barrier()
    return qiskitCir

In [65]:
width = 5 # num_qubits
depth = 5 # number of layers in the circuit (easy + hard)
singleGateSet = ['X', 'H', 'Z', 'I', 'S']
doubleGateSet = ['CNOT_C', 'CNOT_T']
twirlingGateSet = ['X','Y' 'Z', 'I']

circuit = generateCliffordCircuit(width, depth, singleGateSet, doubleGateSet)
qiskitCircuit = transpileListToQiskitCircuit(circuit)
print(circuit)
if depth < 10:
    print(qiskitCircuit)

[['X', 'Z', 'H', 'X', 'ID'], ['CNOT_C', 'I', 'CNOT_T', 'I', 'I'], ['Z', 'Z', 'X', 'Z', 'Z'], ['I', 'CNOT_C', 'CNOT_T', 'I', 'I'], ['S', 'X', 'H', 'ID', 'S']]
     ┌───┐ ░       ░ ┌───┐ ░       ░ ┌───┐
q_0: ┤ X ├─░───■───░─┤ Z ├─░───────░─┤ S ├
     ├───┤ ░   │   ░ ├───┤ ░       ░ ├───┤
q_1: ┤ Z ├─░───┼───░─┤ Z ├─░───■───░─┤ X ├
     ├───┤ ░ ┌─┴─┐ ░ ├───┤ ░ ┌─┴─┐ ░ ├───┤
q_2: ┤ H ├─░─┤ X ├─░─┤ X ├─░─┤ X ├─░─┤ H ├
     ├───┤ ░ └───┘ ░ ├───┤ ░ └───┘ ░ └───┘
q_3: ┤ X ├─░───────░─┤ Z ├─░───────░──────
     └───┘ ░       ░ ├───┤ ░       ░ ┌───┐
q_4: ──────░───────░─┤ Z ├─░───────░─┤ S ├
           ░       ░ └───┘ ░       ░ └───┘


In [66]:
def Clifford_Permute(cliff,pauli):
    '''This function permutes the pauli operators according to the clifford group, so it computes 
    P' = C P C^T, where C is a clifford operator and P is a pauli operator.
    Inputs: cliff - A Qiskit Circuit, Clifford, or Gate object.
            pauli - A Qiskit Pauli object.
    Outputs: new_pauli - A Qiskit Pauli object.'''

    return pauli.evolve(cliff,frame="s")

In [67]:
Clifford_Permute(qiskitCircuit, qk.quantum_info.Pauli('X'*width))

Pauli('-YXXYX')

In [68]:
def params_list(operation_set, pauli_set=["X","Y","Z","I"]):
    '''This function generates a dictionary of parameters for the given operation set.
    Inputs: operation_set - A list of operations.
            pauli_set - A list of pauli operators.
    Outputs: params - A dictionary of parameters, with each set to 0.'''

    params = {}
    for operation in operation_set:
        for pauli in pauli_set:
            params[(operation, pauli)] = 0
    return params

print(params_list(singleGateSet))

{('X', 'X'): 0, ('X', 'Y'): 0, ('X', 'Z'): 0, ('X', 'I'): 0, ('H', 'X'): 0, ('H', 'Y'): 0, ('H', 'Z'): 0, ('H', 'I'): 0, ('Z', 'X'): 0, ('Z', 'Y'): 0, ('Z', 'Z'): 0, ('Z', 'I'): 0, ('ID', 'X'): 0, ('ID', 'Y'): 0, ('ID', 'Z'): 0, ('ID', 'I'): 0, ('S', 'X'): 0, ('S', 'Y'): 0, ('S', 'Z'): 0, ('S', 'I'): 0}


In [71]:
def remove_negatives(pauli):
    '''This function takes a pauli operator and removes the negative sign from it.'''

    if pauli.to_label()[0] == "-":
        return qk.quantum_info.Pauli(pauli.to_label()[1:])
    else:
        return pauli

In [79]:
def pauli_hist(circuit, pauli_initial, params_list=params_list(singleGateSet)):
    '''This function computes the histogram of pauli operators after the circuit has been applied.
    Inputs: circuit - A Qiskit Circuit, Clifford, or Gate object.
            pauli_initial - A Qiskit Pauli object.
            params_list - A dictionary of parameters for the circuit.
    Outputs: pauli_counts - A dictionary of pauli operators and their counts.'''

    pauli = pauli_initial

    for op_instructions in circuit.data:
        if op_instructions.operation.name.upper() == "ID" or op_instructions.operation.name.upper() == "I" or op_instructions.operation.name.upper() == "BARRIER":
            print("skipped ", op_instructions.operation.name.upper(), " operation")
            break
        else:
            pauli = remove_negatives(pauli)
            params_list[(op_instructions.operation.name.upper(), pauli.to_label())] += 1
            pauli = Clifford_Permute(op_instructions.operation, pauli)

    return params_list

print(pauli_hist(qiskitCircuit, qk.quantum_info.Pauli('X')))

skipped  BARRIER  operation
{('X', 'X'): 1, ('X', 'Y'): 0, ('X', 'Z'): 1, ('X', 'I'): 0, ('H', 'X'): 1, ('H', 'Y'): 0, ('H', 'Z'): 0, ('H', 'I'): 0, ('Z', 'X'): 1, ('Z', 'Y'): 0, ('Z', 'Z'): 0, ('Z', 'I'): 0, ('ID', 'X'): 0, ('ID', 'Y'): 0, ('ID', 'Z'): 0, ('ID', 'I'): 0, ('S', 'X'): 0, ('S', 'Y'): 0, ('S', 'Z'): 0, ('S', 'I'): 0}
